In [ ]:
import math
import csv
import pandas as pd
import numpy as np
from scipy.stats import nakagami
from sklearn.cluster import KMeans

# Constants
frequency = 2e9
c = 3e8
a = 9.16
b = 0.16
attenuation_LoS = 1.0
attenuation_NLoS = 20

# Load TSBS locations from CSV file
def load_tsbs_locations(tsbs_csv):
    tsbs_df = pd.read_csv(tsbs_csv)
    tsbs_df['tsbs_index'] = range(1, len(tsbs_df) + 1)
    return tsbs_df[['tsbs_index', 'latitude', 'longitude']]

# Generate child UAV locations using K-means clustering
def generate_child_uav_locations(tsbs_df, num_uavs):
    # Perform K-means clustering on TSBS locations to generate child UAV locations
    kmeans = KMeans(n_clusters=num_uavs, random_state=42)
    kmeans.fit(tsbs_df[['latitude', 'longitude']])
    centroids = kmeans.cluster_centers_

    # Randomly assign altitudes in the range of 300 to 800
    altitudes = np.random.randint(300, 801, size=num_uavs)

    # Create DataFrame for child UAV locations
    child_uav_df = pd.DataFrame({'latitude': centroids[:, 0],
                                 'longitude': centroids[:, 1],
                                 'altitude': altitudes})

    child_uav_df['uav_index'] = range(1, num_uavs + 1)
    child_uav_csv = 'child_uav_locations.csv'
    child_uav_df.to_csv(child_uav_csv, index=False)

    return child_uav_df[['uav_index', 'latitude', 'longitude', 'altitude']]

# Haversine distance calculation
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in km

    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

    distance = R * c  # Distance in kilometers
    return distance * 1000  # Convert to meters

# Path loss calculation
def path_loss(distance, elevation_angle):
    wavelength = c / frequency

    # Check if the distance is zero (same location), set LoS probability to 1
    if distance == 0:
        PLoS = 1
    else:
        # Ensure that the argument of math.exp is within a reasonable range
        arg = -b * (elevation_angle - a)

        PLoS = 1 / (1 + a * math.exp(arg))

    PNLoS = 1 - PLoS

    # Ensure the argument of math.log10 is never zero
    if distance == 0:
        path_loss_dB = 0
    else:
        path_loss_dB = (20 * math.log10(4 * math.pi * distance / wavelength)) + PLoS * attenuation_LoS + PNLoS * attenuation_NLoS

    return path_loss_dB, PLoS, PNLoS

# Calculate elevation angle between TSBS and child-UAV
def calculate_elevation_angle(distance, altitude):
    return math.atan(altitude / distance)

# Calculate transmit power in dBm from watts
def watts_to_dbm(watts):
    return 10 * math.log10(watts)

# Calculate fading in dB
def calculate_fading(PLoS, PNLoS, m):
    z0 = np.abs(nakagami.rvs(m, size=1))
    z1 = np.abs(nakagami.rvs(m, size=1))
    return PLoS * z0 + PNLoS * z1

# Calculate channel gain in dB
def calculate_channel_gain():
    N = int(1e6)
    fading_coefficients = np.abs((1 / np.sqrt(2)) * (np.random.randn(1, N) + 1j * np.random.randn(1, N)))
    fading_coefficients= fading_coefficients.mean()
    return 10*math.log10(fading_coefficients)

# Load TSBS locations
tsbs_csv = "tsbs_placement_imp.csv"
tsbs_locations = load_tsbs_locations(tsbs_csv)

# Varying max_bandwidth
for max_bandwidth in range(0, 1001, 100):
    # Generate child UAV locations
    num_uavs = 4  # Number of UAVs equal to the number of TSBS
    child_uav_locations = generate_child_uav_locations(tsbs_locations, num_uavs)

    # Generate random data rate values
    data_rates = np.random.choice([20, 40, 60, 80, 100], size=len(tsbs_locations) * len(child_uav_locations))

    # Calculate path loss, fading, and channel gain for each TSBS-child UAV pair and assign data rate
    channel_gain_data = []
    for idx_tsbs, tsbs_row in tsbs_locations.iterrows():
        for idx_uav, child_uav_row in child_uav_locations.iterrows():
            distance = haversine(tsbs_row['latitude'], tsbs_row['longitude'],
                                 child_uav_row['latitude'], child_uav_row['longitude'])
            elevation_angle = calculate_elevation_angle(distance, child_uav_row['altitude'])
            path_loss_dB, PLoS, PNLoS = path_loss(distance, elevation_angle)

            # Calculate fading in dB
            m = 1  # Shape parameter (m = 1 for Rayleigh fading)
            fading_db = watts_to_dbm(calculate_fading(PLoS, PNLoS, m))

            transmit_power_dbm = watts_to_dbm(1.3)  # Transmit power of 1.3 watts converted to dBm
            channel_gain = calculate_channel_gain()
            data_rate = data_rates[idx_tsbs * len(child_uav_locations) + idx_uav]
            channel_gain_data.append(
                (tsbs_row['tsbs_index'], child_uav_row['uav_index'], path_loss_dB, PLoS, PNLoS, channel_gain,
                 data_rate))

    # Save results to CSV
    path_loss_csv = f'path_loss_max_bandwidth_{max_bandwidth}.csv'
    with open(path_loss_csv, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['tsbs_index', 'uav_index', 'path_loss', 'PLoS', 'PNLoS', 'channel_gain', 'data_rate'])
        writer.writerows(channel_gain_data)

    print(f"Completed generation for max_bandwidth = {max_bandwidth}")


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-4-e1bd8f6a9277>:90: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)


Completed generation for max_bandwidth = 0


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-4-e1bd8f6a9277>:90: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)


Completed generation for max_bandwidth = 100


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-4-e1bd8f6a9277>:90: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)


...

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-4-e1bd8f6a9277>:90: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)


Completed generation for max_bandwidth = 700


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-4-e1bd8f6a9277>:90: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)


Completed generation for max_bandwidth = 800


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-4-e1bd8f6a9277>:90: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)


In [ ]:
import pandas as pd
import numpy as np

# Initialize an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['Max Bandwidth', 'Sum Rate (F_s)', 'Bandwidth Consumed', 'Number of 1s in A_updated'])

# Iterate over different max_bandwidth values
for max_bandwidth in range(0, 1001, 100):
    # Load the path_loss.csv file for the current max_bandwidth
    path_loss_csv = f'path_loss_max_bandwidth_{max_bandwidth}.csv'
    channel_gain_df = pd.read_csv(path_loss_csv)
    # Calculate the product of magnitude squared of channel gain and inverse of path-loss for each row
    product_column = (channel_gain_df['channel_gain'] ** 2) / (channel_gain_df['path_loss'])

    # Add the product column to the DataFrame
    channel_gain_df['parameter'] = product_column

    # Convert parameter column from dB to watt
    channel_gain_df['parameter'] = 10**(channel_gain_df['parameter'] / 10)

    # Convert interference threshold from watts to dBm
    interference_threshold_dbm = 1.1943**-14  # Convert watts to dBm

    # Calculate optimal power using the division of the 'product' column and interference threshold in dB
    channel_gain_df['optimal_power'] = interference_threshold_dbm / channel_gain_df['parameter']

    def calculate_fading(PLoS, PNLoS, m1,m2):
      z0 = np.abs(nakagami.rvs(m1, size=1))
      z1 = np.abs(nakagami.rvs(m1, size=1))
      z0=10*math.log10(z0)
      z1=10*math.log10(z1)
      return PLoS * z0 + PNLoS * z1

    # Update the calculate_fading function to consider Nakagami distribution
    def calculate_received_power(optimal_power, path_loss, PLoS, PNLoS, m1, m2):
        fading = calculate_fading(PLoS, PNLoS, m1, m2)
        received_power_dB = 10 * np.log10(optimal_power) + fading - path_loss
        # Convert received power from dB to watts
        return received_power_dB

    # Define shape parameter 'm' for Nakagami distribution
    m1 = 1  # For Rayleigh fading, set m = 1
    m2 = 4

    # Calculate received power in watts using the formula
    channel_gain_df['received_power_dB'] = calculate_received_power(channel_gain_df['optimal_power'],
                                                                      channel_gain_df['path_loss'],
                                                                      channel_gain_df['PLoS'],
                                                                      channel_gain_df['PNLoS'],
                                                                      m1, m2)
    noise_power_dbm = -125  # Noise power in dBm

    # Calculate SINR for each row
    def calculate_sinr(received_power, interference_sum, noise_power):
        return received_power / (noise_power + interference_sum)

    # Calculate interference sum for each row (sum of interference powers from other child-UAVs)
    def calculate_interference_sum(channel_gain_df):
        interference_sum = channel_gain_df.groupby(['tsbs_index'])['received_power_dB'].transform('sum') - channel_gain_df['received_power_dB']
        return interference_sum

    # Calculate interference sum column
    channel_gain_df['interference_sum'] = calculate_interference_sum(channel_gain_df)

    # Calculate SINR column
    channel_gain_df['SINR'] = calculate_sinr(channel_gain_df['received_power_dB'], channel_gain_df['interference_sum'], noise_power_dbm)
    # Calculate the bandwidth using the formula: data_rate / log2(1 + SINR)
    bandwidth_column = channel_gain_df['data_rate'] / np.log2(1 + channel_gain_df['SINR'])

    # Add the bandwidth column to the DataFrame
    channel_gain_df['bandwidth'] = bandwidth_column
    spectral_efficiency_column = channel_gain_df['data_rate'] / channel_gain_df['bandwidth']

    # Add the spectral efficiency column to the DataFrame
    channel_gain_df['spectral_efficiency'] = spectral_efficiency_column

    # Save the updated DataFrame to channel_gain.csv
    channel_gain_df.to_csv(path_loss_csv, index=False)


    # Load the data from the CSV file
    df = pd.read_csv(path_loss_csv)

    # Find unique 'sbs_index' and 'uav_index'
    unique_tsbs_index = df['tsbs_index'].unique()
    unique_uav_index = df['uav_index'].unique()

    # Initialize the matrix A with zeros
    A = pd.DataFrame(0, index=unique_tsbs_index, columns=unique_uav_index)

    # Minimum SINR threshold
    min_sinr = -10

    # Iterate over each unique 'sbs_index'
    for tsbs_index in unique_tsbs_index:
        # Filter rows for the current 'sbs_index'
        tsbs_data = df[df['tsbs_index'] == tsbs_index]

        # Find the child-UAV providing maximum SINR
        max_sinr_row = tsbs_data.loc[tsbs_data['SINR'].idxmax()]
        max_sinr_uav_index = max_sinr_row['uav_index']
        max_sinr_value = max_sinr_row['SINR']

        # Check if SINR exceeds the minimum SINR threshold
        if max_sinr_value > min_sinr:
            A.loc[tsbs_index, max_sinr_uav_index] = 1

    # Initialize the matrix A_updated with zeros
    A_updated = A.copy()

    # Initialize counters
    num_links_updated = 0
    C_bandwidth = 0

    max_links = 7

    # Iterate over each child-UAV
    for uav_index in unique_uav_index:
        existing_links = A_updated.index[A_updated[uav_index] == 1]

        # Get data for the current child-UAV
        uav_data = df[df['uav_index'] == uav_index]

        # Initialize counters
        C_bandwidth = 0
        num_links_updated = 0

        # Iterate through TSBSs until maximum resources are utilized
        for tsbs_index in uav_data['tsbs_index'].unique():
            # Filter data for the current tsbs_index and sort by spectral efficiency
            tsbs_data = uav_data[uav_data['tsbs_index'] == tsbs_index]
            tsbs_data = tsbs_data.sort_values(by='spectral_efficiency', ascending=False)

            # Iterate through sorted data
            for _, row in tsbs_data.iterrows():
                bandwidth = row['bandwidth']

                # Check if adding the bandwidth exceeds maximum allowed bandwidth or maximum number of links
                if C_bandwidth + bandwidth <= max_bandwidth and num_links_updated < max_links:
                    # Update matrix A and counters
                    if tsbs_index in existing_links:
                        A_updated.loc[tsbs_index, uav_index] = 1
                        C_bandwidth += bandwidth

                else:
                    # If adding bandwidth exceeds the maximum allowed, break the loop
                    A_updated.loc[tsbs_index, uav_index] = 0
                    break

    # Ensure each column (UAV) has no more than 7 ones (1s)
    for uav_index in unique_uav_index:
        column_sum = A_updated[uav_index].sum()
        if column_sum > max_links:
            # Find indices of extra ones and set them to zero
            extra_indices = A_updated.index[A_updated[uav_index] == 1][max_links:]
            A_updated.loc[extra_indices, uav_index] = 0


    # Calculate sum rate F_s
    F_s = 0
    for tsbs_index in unique_tsbs_index:
        for uav_index in unique_uav_index:
            if A_updated.loc[tsbs_index, uav_index] == 1:
                data_rate = df[(df['tsbs_index'] == tsbs_index) & (df['uav_index'] == uav_index)]['data_rate'].values[0]
                F_s += data_rate

    # Initialize C' as the total number of associated TSBSs
    C_prime = A_updated.sum().sum()

    # Initialize Cb as the cumulative bandwidth used by associated TSBSs
    C_b = (A_updated * df['bandwidth']).sum().sum()

    # Initialize Fs as total sum-rate of associated TSBSs
    Fs = F_s

    # Initialize RB (required bandwidth) as 1.6 Gbps
    RB = 1.6 * 1000  # Convert Gbps to Mbps

    # Continue looping until Fs is less than or equal to RB
    while Fs > RB:
        # Select child-UAV with max associated TSBSs
        max_tsbs_uav = A_updated.sum(axis=0).idxmax()

        # Select TSBS with minimum data rate demand
        min_data_rate_tsbs = df.loc[df['uav_index'] == max_tsbs_uav].sort_values(by='data_rate').iloc[0]
        min_data_rate_tsbs_index = min_data_rate_tsbs['tsbs_index']
        min_data_rate_data_rate = min_data_rate_tsbs['data_rate']

        # De-associate the selected pair and update matrix A, decrement C', update Fs, and decrement Cb
        A_updated.loc[min_data_rate_tsbs_index, max_tsbs_uav] = 0
        C_prime -= 1
        Fs -= min_data_rate_data_rate
        C_b -= min_data_rate_tsbs['bandwidth']

    # Calculate the number of 1s in A_updated
    num_ones = A_updated.values.sum()

    # Append results to the results DataFrame
    results_df = results_df.append({'Max Bandwidth': max_bandwidth,
                                    'Sum Rate (F_s)': F_s,
                                    'Bandwidth Consumed': C_b,
                                    'Number of 1s in A_updated': num_ones}, ignore_index=True)

# Save results to CSV
results_df.to_csv('results.csv', index=False)
print("Results saved to results.csv")


<ipython-input-7-42928563d2ce>:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  z0=10*math.log10(z0)
<ipython-input-7-42928563d2ce>:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  z1=10*math.log10(z1)
<ipython-input-7-42928563d2ce>:199: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Max Bandwidth': max_bandwidth,
<ipython-input-7-42928563d2ce>:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before per

Results saved to results.csv


<ipython-input-7-42928563d2ce>:199: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Max Bandwidth': max_bandwidth,
<ipython-input-7-42928563d2ce>:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  z0=10*math.log10(z0)
<ipython-input-7-42928563d2ce>:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  z1=10*math.log10(z1)
<ipython-input-7-42928563d2ce>:199: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Max

In [ ]:
!pip install scikit-learn-extra
import math
import csv
import pandas as pd
import numpy as np
from scipy.stats import nakagami
from sklearn_extra.cluster import KMedoids  # Import KMedoids from sklearn_extra

# Constants
frequency = 2e9
c = 3e8
a = 9.16
b = 0.16
attenuation_LoS = 1.0
attenuation_NLoS = 20

# Load TSBS locations from CSV file
def load_tsbs_locations(tsbs_csv):
    tsbs_df = pd.read_csv(tsbs_csv)
    tsbs_df['tsbs_index'] = range(1, len(tsbs_df) + 1)
    return tsbs_df[['tsbs_index', 'latitude', 'longitude']]

# Generate child UAV locations using K-medoids clustering (PAM)
def generate_child_uav_locations(tsbs_df, num_uavs):
    # Perform K-medoids clustering on TSBS locations to generate child UAV locations
    kmedoids = KMedoids(n_clusters=num_uavs, random_state=42)
    kmedoids.fit(tsbs_df[['latitude', 'longitude']])
    centroids_idx = kmedoids.medoid_indices_
    centroids = tsbs_df[['latitude', 'longitude']].iloc[centroids_idx].values

    # Randomly assign altitudes in the range of 300 to 800
    altitudes = np.random.randint(300, 801, size=num_uavs)

    # Create DataFrame for child UAV locations
    child_uav_df = pd.DataFrame({'latitude': centroids[:, 0],
                                 'longitude': centroids[:, 1],
                                 'altitude': altitudes})

    child_uav_df['uav_index'] = range(1, num_uavs + 1)
    child_uav_csv = 'child_uav_locations_medoids.csv'
    child_uav_df.to_csv(child_uav_csv, index=False)

    return child_uav_df[['uav_index', 'latitude', 'longitude', 'altitude']]

# Haversine distance calculation
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in km

    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

    distance = R * c  # Distance in kilometers
    return distance * 1000  # Convert to meters

# Path loss calculation
def path_loss(distance, elevation_angle):
    wavelength = c / frequency

    # Check if the distance is zero (same location), set LoS probability to 1
    if distance == 0:
        PLoS = 1
    else:
        # Ensure that the argument of math.exp is within a reasonable range
        arg = -b * (elevation_angle - a)

        PLoS = 1 / (1 + a * math.exp(arg))

    PNLoS = 1 - PLoS

    # Ensure the argument of math.log10 is never zero
    if distance == 0:
        path_loss_dB = 0
    else:
        path_loss_dB = (20 * math.log10(4 * math.pi * distance / wavelength)) + PLoS * attenuation_LoS + PNLoS * attenuation_NLoS

    return path_loss_dB, PLoS, PNLoS

# Calculate elevation angle between TSBS and child-UAV
def calculate_elevation_angle(distance, altitude):
    return math.atan(altitude / distance)

# Calculate transmit power in dBm from watts
def watts_to_dbm(watts):
    return 10 * math.log10(watts)

# Calculate fading in dB
def calculate_fading(PLoS, PNLoS, m):
    z0 = np.abs(nakagami.rvs(m, size=1))
    z1 = np.abs(nakagami.rvs(m, size=1))
    return PLoS * z0 + PNLoS * z1

# Calculate channel gain in dB
def calculate_channel_gain():
    N = int(1e6)
    fading_coefficients = np.abs((1 / np.sqrt(2)) * (np.random.randn(1, N) + 1j * np.random.randn(1, N)))
    fading_coefficients= fading_coefficients.mean()
    return 10*math.log10(fading_coefficients)

# Load TSBS locations
tsbs_csv = "tsbs_placement_imp.csv"
tsbs_locations = load_tsbs_locations(tsbs_csv)

# Varying max_bandwidth
for max_bandwidth in range(0, 1001, 100):
    # Generate child UAV locations
    num_uavs = 4  # Number of UAVs equal to the number of TSBS
    child_uav_locations = generate_child_uav_locations(tsbs_locations, num_uavs)

    # Generate random data rate values
    data_rates = np.random.choice([20, 40, 60, 80, 100], size=len(tsbs_locations) * len(child_uav_locations))

    # Calculate path loss, fading, and channel gain for each TSBS-child UAV pair and assign data rate
    channel_gain_data = []
    for idx_tsbs, tsbs_row in tsbs_locations.iterrows():
        for idx_uav, child_uav_row in child_uav_locations.iterrows():
            distance = haversine(tsbs_row['latitude'], tsbs_row['longitude'],
                                 child_uav_row['latitude'], child_uav_row['longitude'])
            elevation_angle = calculate_elevation_angle(distance, child_uav_row['altitude'])
            path_loss_dB, PLoS, PNLoS = path_loss(distance, elevation_angle)

            # Filter out rows with path_loss_dB of 0 for the specific tsbs_index
            if path_loss_dB != 0:
                # Calculate fading in dB
                m = 1  # Shape parameter (m = 1 for Rayleigh fading)
                fading_db = watts_to_dbm(calculate_fading(PLoS, PNLoS, m))

                transmit_power_dbm = watts_to_dbm(1.3)  # Transmit power of 1.3 watts converted to dBm
                channel_gain = calculate_channel_gain()
                data_rate = data_rates[idx_tsbs * len(child_uav_locations) + idx_uav]
                channel_gain_data.append(
                    (tsbs_row['tsbs_index'], child_uav_row['uav_index'], path_loss_dB, PLoS, PNLoS, channel_gain,
                     data_rate))

    # Save results to CSV
    path_loss_csv = f'path_loss_max_bandwidth_{max_bandwidth}.csv'
    with open(path_loss_csv, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['tsbs_index', 'uav_index', 'path_loss', 'PLoS', 'PNLoS', 'channel_gain', 'data_rate'])
        writer.writerows(channel_gain_data)

    print(f"Completed generation for max_bandwidth = {max_bandwidth}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.3 MB/s eta 0:00:00


<ipython-input-3-a027b6a3759f>:92: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)
<ipython-input-3-a027b6a3759f>:88: RuntimeWarning: divide by zero encountered in scalar divide
  return math.atan(altitude / distance)


Completed generation for max_bandwidth = 0


<ipython-input-3-a027b6a3759f>:92: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)
<ipython-input-3-a027b6a3759f>:88: RuntimeWarning: divide by zero encountered in scalar divide
  return math.atan(altitude / distance)


Completed generation for max_bandwidth = 100


...

Completed generation for max_bandwidth = 800


<ipython-input-3-a027b6a3759f>:92: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)
<ipython-input-3-a027b6a3759f>:88: RuntimeWarning: divide by zero encountered in scalar divide
  return math.atan(altitude / distance)


Completed generation for max_bandwidth = 900


<ipython-input-3-a027b6a3759f>:92: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)
<ipython-input-3-a027b6a3759f>:88: RuntimeWarning: divide by zero encountered in scalar divide
  return math.atan(altitude / distance)


Completed generation for max_bandwidth = 1000


In [ ]:
import pandas as pd
import numpy as np

# Initialize an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['Max Bandwidth', 'Sum Rate (F_s)', 'Bandwidth Consumed', 'Number of 1s in A_updated'])

# Iterate over different max_bandwidth values
for max_bandwidth in range(0, 1001, 100):
    # Load the path_loss.csv file for the current max_bandwidth
    path_loss_csv = f'path_loss_max_bandwidth_{max_bandwidth}.csv'
    channel_gain_df = pd.read_csv(path_loss_csv)
    # Calculate the product of magnitude squared of channel gain and inverse of path-loss for each row
    product_column = (channel_gain_df['channel_gain'] ** 2) / (channel_gain_df['path_loss'])

    # Add the product column to the DataFrame
    channel_gain_df['parameter'] = product_column

    # Convert parameter column from dB to watt
    channel_gain_df['parameter'] = 10**(channel_gain_df['parameter'] / 10)

    # Convert interference threshold from watts to dBm
    interference_threshold_dbm = 1.1943**-14  # Convert watts to dBm

    # Calculate optimal power using the division of the 'product' column and interference threshold in dB
    channel_gain_df['optimal_power'] = interference_threshold_dbm / channel_gain_df['parameter']

    def calculate_fading(PLoS, PNLoS, m1,m2):
      z0 = np.abs(nakagami.rvs(m1, size=1))
      z1 = np.abs(nakagami.rvs(m1, size=1))
      z0=10*math.log10(z0)
      z1=10*math.log10(z1)
      return PLoS * z0 + PNLoS * z1

    # Update the calculate_fading function to consider Nakagami distribution
    def calculate_received_power(optimal_power, path_loss, PLoS, PNLoS, m1, m2):
        fading = calculate_fading(PLoS, PNLoS, m1, m2)
        received_power_dB = 10 * np.log10(optimal_power) + fading - path_loss
        # Convert received power from dB to watts
        return received_power_dB

    # Define shape parameter 'm' for Nakagami distribution
    m1 = 1  # For Rayleigh fading, set m = 1
    m2 = 4

    # Calculate received power in watts using the formula
    channel_gain_df['received_power_dB'] = calculate_received_power(channel_gain_df['optimal_power'],
                                                                      channel_gain_df['path_loss'],
                                                                      channel_gain_df['PLoS'],
                                                                      channel_gain_df['PNLoS'],
                                                                      m1, m2)
    noise_power_dbm = -125  # Noise power in dBm

    # Calculate SINR for each row
    def calculate_sinr(received_power, interference_sum, noise_power):
        return received_power / (noise_power + interference_sum)

    # Calculate interference sum for each row (sum of interference powers from other child-UAVs)
    def calculate_interference_sum(channel_gain_df):
        interference_sum = channel_gain_df.groupby(['tsbs_index'])['received_power_dB'].transform('sum') - channel_gain_df['received_power_dB']
        return interference_sum

    # Calculate interference sum column
    channel_gain_df['interference_sum'] = calculate_interference_sum(channel_gain_df)

    # Calculate SINR column
    channel_gain_df['SINR'] = calculate_sinr(channel_gain_df['received_power_dB'], channel_gain_df['interference_sum'], noise_power_dbm)
    # Calculate the bandwidth using the formula: data_rate / log2(1 + SINR)
    bandwidth_column = channel_gain_df['data_rate'] / np.log2(1 + channel_gain_df['SINR'])

    # Add the bandwidth column to the DataFrame
    channel_gain_df['bandwidth'] = bandwidth_column
    spectral_efficiency_column = channel_gain_df['data_rate'] / channel_gain_df['bandwidth']

    # Add the spectral efficiency column to the DataFrame
    channel_gain_df['spectral_efficiency'] = spectral_efficiency_column

    # Save the updated DataFrame to channel_gain.csv
    channel_gain_df.to_csv(path_loss_csv, index=False)


    # Load the data from the CSV file
    df = pd.read_csv(path_loss_csv)

    # Find unique 'sbs_index' and 'uav_index'
    unique_tsbs_index = df['tsbs_index'].unique()
    unique_uav_index = df['uav_index'].unique()

    # Initialize the matrix A with zeros
    A = pd.DataFrame(0, index=unique_tsbs_index, columns=unique_uav_index)

    # Minimum SINR threshold
    min_sinr = -10

    # Iterate over each unique 'sbs_index'
    for tsbs_index in unique_tsbs_index:
        # Filter rows for the current 'sbs_index'
        tsbs_data = df[df['tsbs_index'] == tsbs_index]

        # Find the child-UAV providing maximum SINR
        max_sinr_row = tsbs_data.loc[tsbs_data['SINR'].idxmax()]
        max_sinr_uav_index = max_sinr_row['uav_index']
        max_sinr_value = max_sinr_row['SINR']

        # Check if SINR exceeds the minimum SINR threshold
        if max_sinr_value > min_sinr:
            A.loc[tsbs_index, max_sinr_uav_index] = 1

    # Initialize the matrix A_updated with zeros
    A_updated = A.copy()

    # Initialize counters
    num_links_updated = 0
    C_bandwidth = 0

    max_links = 7

    # Iterate over each child-UAV
    for uav_index in unique_uav_index:
        existing_links = A_updated.index[A_updated[uav_index] == 1]

        # Get data for the current child-UAV
        uav_data = df[df['uav_index'] == uav_index]

        # Initialize counters
        C_bandwidth = 0
        num_links_updated = 0

        # Iterate through TSBSs until maximum resources are utilized
        for tsbs_index in uav_data['tsbs_index'].unique():
            # Filter data for the current tsbs_index and sort by spectral efficiency
            tsbs_data = uav_data[uav_data['tsbs_index'] == tsbs_index]
            tsbs_data = tsbs_data.sort_values(by='spectral_efficiency', ascending=False)

            # Iterate through sorted data
            for _, row in tsbs_data.iterrows():
                bandwidth = row['bandwidth']

                # Check if adding the bandwidth exceeds maximum allowed bandwidth or maximum number of links
                if C_bandwidth + bandwidth <= max_bandwidth and num_links_updated < max_links:
                    # Update matrix A and counters
                    if tsbs_index in existing_links:
                        A_updated.loc[tsbs_index, uav_index] = 1
                        C_bandwidth += bandwidth

                else:
                    # If adding bandwidth exceeds the maximum allowed, break the loop
                    A_updated.loc[tsbs_index, uav_index] = 0
                    break

    # Ensure each column (UAV) has no more than 7 ones (1s)
    for uav_index in unique_uav_index:
        column_sum = A_updated[uav_index].sum()
        if column_sum > max_links:
            # Find indices of extra ones and set them to zero
            extra_indices = A_updated.index[A_updated[uav_index] == 1][max_links:]
            A_updated.loc[extra_indices, uav_index] = 0


    # Calculate sum rate F_s
    F_s = 0
    for tsbs_index in unique_tsbs_index:
        for uav_index in unique_uav_index:
            if A_updated.loc[tsbs_index, uav_index] == 1:
                data_rate = df[(df['tsbs_index'] == tsbs_index) & (df['uav_index'] == uav_index)]['data_rate'].values[0]
                F_s += data_rate

    # Initialize C' as the total number of associated TSBSs
    C_prime = A_updated.sum().sum()

    # Initialize Cb as the cumulative bandwidth used by associated TSBSs
    C_b = (A_updated * df['bandwidth']).sum().sum()

    # Initialize Fs as total sum-rate of associated TSBSs
    Fs = F_s

    # Initialize RB (required bandwidth) as 1.6 Gbps
    RB = 1.6 * 1000  # Convert Gbps to Mbps

    # Continue looping until Fs is less than or equal to RB
    while Fs > RB:
        # Select child-UAV with max associated TSBSs
        max_tsbs_uav = A_updated.sum(axis=0).idxmax()

        # Select TSBS with minimum data rate demand
        min_data_rate_tsbs = df.loc[df['uav_index'] == max_tsbs_uav].sort_values(by='data_rate').iloc[0]
        min_data_rate_tsbs_index = min_data_rate_tsbs['tsbs_index']
        min_data_rate_data_rate = min_data_rate_tsbs['data_rate']

        # De-associate the selected pair and update matrix A, decrement C', update Fs, and decrement Cb
        A_updated.loc[min_data_rate_tsbs_index, max_tsbs_uav] = 0
        C_prime -= 1
        Fs -= min_data_rate_data_rate
        C_b -= min_data_rate_tsbs['bandwidth']

    # Calculate the number of 1s in A_updated
    num_ones = A_updated.values.sum()

    # Append results to the results DataFrame
    results_df = results_df.append({'Max Bandwidth': max_bandwidth,
                                    'Sum Rate (F_s)': F_s,
                                    'Bandwidth Consumed': C_b,
                                    'Number of 1s in A_updated': num_ones}, ignore_index=True)

# Save results to CSV
results_df.to_csv('results_medoid_ny.csv', index=False)
print("Results saved to results.csv")


<ipython-input-2-717be259b282>:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  z0=10*math.log10(z0)
<ipython-input-2-717be259b282>:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  z1=10*math.log10(z1)
<ipython-input-2-717be259b282>:199: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Max Bandwidth': max_bandwidth,
<ipython-input-2-717be259b282>:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before per

Results saved to results.csv


<ipython-input-2-717be259b282>:199: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Max Bandwidth': max_bandwidth,
